API: https://rapidapi.com/SAdrian/api/moviesdatabase

In [26]:
#Importamos las librerias necesarias
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
from time import sleep

#importamos librerias de selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

# Configuraciones
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

In [27]:
#creamos el diccionario donde alamcenaremos la información
dic_API_Pelis = {'Tipo':[],
                 'Nombre':[],
                 'Año_estreno':[],
                 'Mes_estreno': [],
                 'Id':[],
                 'Genero':[]}

# creamos una lista para almacenar las respuestas de la funcion de llamada a la API
lista_respuestas = []
lista_info_respuestas = []


#creamos las variables relacionadas con el ENDPOINT
url = "https://moviesdatabase.p.rapidapi.com/titles"

# API_key Nuria = '58296df2a9mshb28a81ec4003c84p15333ejsn11ad25c7c1b8'
# API_key Sharon = '2d7083a699mshd0e8039e2063bedp140338jsn9b141653fce1'
# API_key3 Silvia = '039163dca0msh7ecfddf47a57b42p193f39jsn80c27500a5c2'

# headers = {"X-RapidAPI-Key": API_key,"X-RapidAPI-Host": "moviesdatabase.p.rapidapi.com"}

titleType_list = ['movie', 'short']
genere_list = ['Action']
               #'Drama', 'Comedy']

              # "year":"2010"

# querystring = {"titleType":"short","startYear":"2010","endYear":"2024","genre":"Drama"}

In [28]:
# Definimos una fucnción para extraer información de la API y almacenarla en un diccionario

def llamar_API (url,type, genere, **dict):
    '''
    Esta función realiza una llamada a una API a partir de una url y guarda la información en un diccionario.

    Parametros:
    -----------
    url (str): la dirección url de la API a la que queremos hacer la llamada
    type (str): El tipo de pelícua que queremos buscar. 
    genere (dtr): El género que ueremos buscar. 
    dict (dict): el diccionario dónde se almacenará la información

    Returns:
    -------
    dict (dict): el diccionario donde se almacenará la información
    '''
    #definimos la querystring y la API key
    querystring = {"titleType":type,"year":"2010","genre":genere}

    # API_key = '2d7083a699mshd0e8039e2063bedp140338jsn9b141653fce1'
    # API_key Nuria = '58296df2a9mshb28a81ec4003c84p15333ejsn11ad25c7c1b8'
    # API_key = '2d7083a699mshd0e8039e2063bedp140338jsn9b141653fce1'
    API_key = '039163dca0msh7ecfddf47a57b42p193f39jsn80c27500a5c2'
    headers = {"X-RapidAPI-Key": API_key,"X-RapidAPI-Host": "moviesdatabase.p.rapidapi.com"}    

    #iniciamos el loop while para navegar por las distintas páginas
    while url.startswith('https'):

        #realizmos la llamada y almacenamos los códigos de respuesta en la lista de respuestas
        response_pelis = requests.get(url, headers=headers, params=querystring)
        lista_respuestas.append(response_pelis.status_code)
        lista_info_respuestas.append(response_pelis.reason)

        if response_pelis.status_code == 200:

            #convertimos los resulatos a formato json
            js_response_pelis = response_pelis.json()

            #guardamos la inforación de la API dónde tenemos la dirección de la siguiente página
            next = js_response_pelis['next']

            #guardamos la parte donde tenemos la infomación en una variable
            info_pelis = js_response_pelis['results']

            #guardamos toda la información en el diccionario con un for loop    
            for peli in info_pelis:

                dict['Genero'].append(genere)
                dict['Tipo'].append(peli['titleType']['text'])
                dict['Nombre'].append(peli['titleText']['text'])
                dict['Id'].append(peli['id'])
                dict['Año_estreno'].append(peli['releaseYear']['year'])

                #Hacemos un try/expect para las películas que no se han estrenado todavía            
                try:
                    dict['Mes_estreno'].append(peli['releaseDate']['month'])
                except:            
                    dict['Mes_estreno'].append('por estrenar')

            #iniciamos el loop try-except para que cree la variable de la nueva url mientras tengamos una url en ´next´
            try:
                #definimos la base de la nueva url
                url_fixed = "https://moviesdatabase.p.rapidapi.com"
                #definimos la nueva url
                url=url_fixed+next
                #definimos el nuevo querystring
                querystring = {}

            except:
                break

        else:
            print(f'Error {response_pelis.status_code} en la llamada a la API')
            break


    #mostramos los resultados
    print(f'Fin: Se han añadido l@s {type}s del género {genere} a tu diccionario.')

In [29]:
# Iniciamos el loop for para conseguir información de todos los tipos y géneros
for type in titleType_list:
    for genere in genere_list:
        #iniciamos la funcion que llama a la API
        llamar_API(url,type,genere, **dic_API_Pelis)

Fin: Se han añadido l@s movies del género Action a tu diccionario.
Fin: Se han añadido l@s shorts del género Action a tu diccionario.


In [32]:
df = pd.DataFrame(dic_API_Pelis)
df.head()


,Tipo,Nombre,Año_estreno,Mes_estreno,Id,Genero
0,Movie,Tekken,2010,3,tt0411951,Action
1,Movie,Venus & Vegas,2010,11,tt0423474,Action
2,Movie,The A-Team,2010,6,tt0429493,Action
3,Movie,Segurança Nacional,2010,5,tt0439801,Action
4,Movie,Scott Pilgrim vs. the World,2010,11,tt0446029,Action


In [31]:
df['Año_estreno'].unique()

array([2010])

In [40]:
years = range(2010,2025)
year = years[2]
year

2012